# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from Itinerary_config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [14]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df

,Unnamed: 0,City,Country,Max Temp,Decription,Lat,Lng,Hotel Name
0,0,Kapaa,US,80.58,light rain,22.0752,-159.3190,Pono Kai Resort
1,3,Mogadishu,SO,77.99,scattered clouds,2.0371,45.3438,SYL Hotel
2,4,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Page 63 hostel
3,5,Tual,ID,77.67,overcast clouds,-5.6667,132.7500,Suita
4,8,Brownsville,US,74.08,broken clouds,25.9018,-97.4975,Cameron Hotel
...,...,...,...,...,...,...,...,...
383,737,Singaraja,ID,85.19,broken clouds,-8.1120,115.0882,Hotel Sentral
384,742,Hammond,US,39.49,overcast clouds,41.5834,-87.5000,Fairfield Inn & Suites by Marriott Chicago Sou...
385,743,Grand Gaube,MU,76.37,clear sky,-20.0064,57.6608,Coco Villa
386,744,Nabire,ID,75.40,overcast clouds,-3.3667,135.4833,Hotel Anggrek


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [37]:
# Configure the map plot

map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Decription"]
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Decription)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [38]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[(vacation_df["City"] == "Saint George")]

vacation_end = vacation_df.loc[(vacation_df["City"] == "North Bend")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Laguna")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Pacifica")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Fortuna")]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [39]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,Unnamed: 0,City,Country,Max Temp,Decription,Lat,Lng,Hotel Name
127,235,Saint George,US,48.79,overcast clouds,37.1041,-113.5841,"The Advenire, an Autograph Collection Hotel"
166,327,Laguna,US,50.05,light rain,38.4210,-121.4238,Holiday Inn Express & Suites
225,439,Pacifica,US,48.63,overcast clouds,37.6138,-122.4869,Holiday Inn Express & Suites Pacifica
133,244,Fortuna,US,44.65,scattered clouds,40.5982,-124.1573,Comfort Inn & Suites Redwood Country
82,141,North Bend,US,44.51,scattered clouds,43.4065,-124.2243,The Mill Casino Hotel


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [40]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lng', '']]

# Display sample data
waypoints_df

,Lng,Lat
127,-113.5841,37.1041
166,-121.4238,38.4210
225,-122.4869,37.6138
133,-124.1573,40.5982
82,-124.2243,43.4065


## Use GeoViews to create map that shows the four cities in the itinerary

In [124]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    scale = 1,
    size = 200,
)

# Display the map plot
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [8]:
# Display the route_map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [58]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [73]:
# Set an empty waypoints String variable
waypoints = str()

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [100]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response.json()

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-113.5841, 37.1041], 'original_index': 0},
     {'location': [-121.4238, 38.421], 'original_index': 1},
     {'location': [-122.4869, 37.6138], 'original_index': 2},
     {'location': [-124.1573, 40.5982], 'original_index': 3},
     {'location': [-124.2243, 43.4065], 'original_index': 4}],
    'units': 'metric',
    'distance': 2089877,
    'distance_units': 'meters',
    'time': 75199.95,
    'legs': [{'distance': 1099114,
      'time': 36425.199,
      'steps': [{'from_index': 0,
        'to_index': 2,
        'distance': 155,
        'time': 55.8,
        'instruction': {'text': 'Drive south.'}},
       {'from_index': 2,
        'to_index': 4,
        'distance': 258,
        'time': 31.938,
        'instruction': {'text': 'Turn left onto South 100 West.'}},
       {'from_index': 4,
        'to_index': 9,
        'distance': 380,
        'time': 29.492,
        'instruction': {'text'

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [93]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response.json()["features"][0]["geometry"]["coordinates"]
legs

[[[-113.584121, 37.104063],
  [-113.584121, 37.103732],
  [-113.585456, 37.103734],
  [-113.585462, 37.103094],
  [-113.585459, 37.101411],
  [-113.587598, 37.101419],
  [-113.588851, 37.101426],
  [-113.588961, 37.101426],
  [-113.589288, 37.101428],
  [-113.589737, 37.10143],
  [-113.58996, 37.101255],
  [-113.589421, 37.100769],
  [-113.589315, 37.100674],
  [-113.589127, 37.100509],
  [-113.588977, 37.100363],
  [-113.588839, 37.100219],
  [-113.588648, 37.100015],
  [-113.588528, 37.099862],
  [-113.588295, 37.09952],
  [-113.58819, 37.099355],
  [-113.588104, 37.09921],
  [-113.588, 37.099003],
  [-113.587937, 37.09884],
  [-113.587875, 37.098698],
  [-113.587815, 37.098548],
  [-113.587723, 37.098312],
  [-113.587607, 37.098014],
  [-113.587494, 37.097684],
  [-113.587419, 37.097472],
  [-113.587304, 37.09718],
  [-113.587278, 37.097109],
  [-113.587137, 37.09673],
  [-113.586989, 37.096331],
  [-113.58683, 37.095874],
  [-113.586801, 37.095791],
  [-113.586731, 37.095583],
  [-

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [105]:
# Create and empty list to store the longitude of each step
step_long = []

# Create and empty list to store the latitude of step
step_lat = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        step_long.append(coordinate[0])
        step_lat.append(coordinate[1])
    

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [110]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame(
    {'Longitude': step_long,
     'Latitude': step_lat,
    })

# Display sample data
route_df.head()

,Longitude,Latitude
0,-113.584121,37.104063
1,-113.584121,37.103732
2,-113.585456,37.103734
3,-113.585462,37.103094
4,-113.585459,37.101411


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [125]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "red"
)

In [126]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]
   .Path.I   :Path   [Longitude,Latitude]